Dataset:
https://dumps.wikimedia.org/backup-index.html

Before Running the parser below, please run WikipediaExtractor script:

python2 WikipediaExtractor.py --json --bytes 10G <wikipedia-file-path>

Download:
https://github.com/attardi/wikiextractor

Source:
https://www.mediawiki.org/wiki/Alternative_parsers

# Load Libraries

In [25]:
import json
import re
import string
import unidecode

# Declare variables

In [2]:
wikipedia_dataset = '../Datasets/ptwiki-20170820-pages-articles-parsed.json'

# Load All Data

In [3]:
data_reader = open(wikipedia_dataset, 'r')

# Parse all articles

In [23]:
# Beginning
data_reader.seek(0)

# Parse all text
%time articles = [json.loads(line)['text'] for line in data_reader]

CPU times: user 19.3 s, sys: 504 ms, total: 19.8 s
Wall time: 19.8 s


# Convert text to token list

In [140]:
def text_preprocessing(text):
    numbers = '0123456789'
    punctuation = re.sub('[!?\.]', '', string.punctuation)
    translator = str.maketrans('', '', punctuation + numbers) 
    
    # Remove punctuation and numbers
    preprocessed = text.translate(translator)
    
    # All words to Upper case
    #preprocessed = preprocessed.upper()
    
    # Remove accentuation
    #preprocessed = unidecode.unidecode(preprocessed)
    
    # Split text in sentences
    sentences = re.split('[!?\.\\n]', preprocessed)
    
    # Remove spaces from start and end of sentence
    sentences = [sentence.strip()
                 for sentence in sentences]
    
    # First caracter of each sentence to lower
    sentences = [sentence[:1].lower() + sentence[1:]
                 for sentence in sentences
                 if sentence and ' '  in sentence]
    
    # Split sentence in tokens
    sentences = [re.split(' +', sentence)
                 for sentence in sentences]    
    
    #sentences = [re.split(' +', sentence.strip()[0].lower() + sentence.strip()[1:])
    #             for sentence in sentences[1:]
    #             if sentence and ' '  in sentence]
    
    return sentences

In [ ]:
sentences = []
for article in articles[:100]:
    sentences += text_preprocessing(article)

In [144]:
len(sentences)

2587

In [ ]:
vocab = list(set(word for sentence in sentences for word in sentence))
vocab_ids = {word:wid for wid, word in enumerate(vocab, 1)}
vocab_ids